In [1]:
import os
import sys
sys.path.append('./')

import args
import argparse
import logging

import torch
from models.AdversarialModel import AdversarialModel
from models.FOCALModules import FOCAL
from models.loss import FOCALLoss
from data.EfficientDataset import MESAPairDataset
from data.Augmentaion import init_augmenter

import datetime
import importlib

/data8/jungmin/anaconda3/envs/patchtst/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
print(f"base_config: \n {args.base_config}")
# print(f"focal_config: \n {args.focal_config} \n")

base_config: 
 {'train_data_dir': '/data8/jungmin/uot_class/MIE1517_DL/FM_for_bio_signal/src/foundation/dataset/pair_small', 'val_data_dir': '/data8/jungmin/uot_class/MIE1517_DL/FM_for_bio_signal/src/foundation/dataset/pair', 'test_data_dir': '/NFS/Users/moonsh/data/mesa/preproc/pair_test', 'modalities': ['ecg', 'hr'], 'label_key': 'stage', 'subject_key': 'subject_idx', 'train_num_subjects': 100, 'test_num_subjects': 50, 'device': device(type='cpu'), 'log_save_dir': '/data8/jungmin/uot_class/MIE1517_DL/FM_for_bio_signal/logs'}


### DataLoader

In [3]:
train_dataset = MESAPairDataset(file_path=args.base_config['train_data_dir'],
                                    modalities=args.base_config['modalities'],
                                    subject_idx=args.base_config['subject_key'],
                                    stage=args.base_config['label_key'])
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                            batch_size=4,
                                            shuffle=True,
                                            num_workers=1)

In [4]:
print(train_dataset.__len__())

for i , (raw_modal_1, raw_modal_2, subj, sleep) in enumerate(train_loader):
    print(i)
    print(raw_modal_1.shape)
    print(raw_modal_2.shape)
    print(subj.shape)
    print(sleep.shape)
    
    break

14
0
torch.Size([4, 7680])
torch.Size([4, 30])
torch.Size([4])
torch.Size([4])


### Augmentation

In [5]:
print(f"data_config: \n {args.data_config} \n")

data_config: 
 {'modalities': ['ecg', 'hr'], 'label_key': 'stage', 'augmenter': 'GaussianNoise', 'augmenter_config': {'GaussianNoise': {'max_noise_std': 0.1}, 'AmplitudeScale': {'amplitude_scale': 0.5}}, 'num_classes': None} 



In [6]:
augmenter = init_augmenter(args)

Loading GaussianNoise augmenter...


In [7]:
raw_modal_1.shape

torch.Size([4, 7680])

In [8]:
# It changes the shape of input: (B, seq) -> (B
augmenter(raw_modal_1).shape

torch.Size([4, 7680])

In [9]:
for i , (raw_modal_1, raw_modal_2, subj, sleep) in enumerate(train_loader):
    aug_modal_1 = augmenter(raw_modal_1)
    aug_modal_2 = augmenter(raw_modal_2)
    print(aug_modal_1.shape)
    print(aug_modal_2.shape)

torch.Size([4, 7680])
torch.Size([4, 30])
torch.Size([4, 7680])
torch.Size([4, 30])
torch.Size([4, 7680])
torch.Size([4, 30])
torch.Size([2, 7680])
torch.Size([2, 30])


### Backbone

In [10]:
import importlib
from models.Backbone import DeepSense
importlib.reload(args)

<module 'args' from '/data8/jungmin/uot_class/MIE1517_DL/FM_for_bio_signal/src/foundation/args.py'>

In [11]:
backbone_model = DeepSense(args)
# dims = [1, 16, 32]

[1, 16, 32]
ecg extractor is initialized.
hr extractor is initialized.
ecg recurrent layer is initialized.
hr recurrent layer is initialized.
** Finished Initializing DeepSense Backbone **


In [12]:
aug_modal_1.shape

torch.Size([2, 7680])

In [13]:
7680/64

120.0

In [14]:
enc_mod_features = backbone_model(aug_modal_1, aug_modal_2)

mod1 cnn feature shape: torch.Size([2, 64, 280]) mod2 cnn feature shape: torch.Size([2, 64, 30])
mod1 rnn feature shape: torch.Size([2, 17920]) mod2 rnn feature shape: torch.Size([2, 1920])


In [19]:
enc_mod_features

{'ecg': tensor([[ 0.0373, -0.0691,  0.0893, -0.1336,  0.0277,  0.0348,  0.0976,  0.0775,
           0.0362, -0.1074,  0.0804, -0.1162,  0.0019,  0.0435,  0.0680, -0.1362,
           0.0716, -0.1131, -0.0832, -0.0562, -0.0432,  0.0282,  0.0135, -0.0318,
          -0.1062,  0.0794, -0.0139,  0.0276,  0.0592,  0.1105,  0.1581,  0.0352,
           0.1384, -0.1379, -0.0503,  0.0486,  0.0564,  0.1320, -0.1415,  0.0065,
          -0.0746, -0.0368, -0.0906,  0.0764, -0.0301,  0.0606,  0.0472, -0.0630,
           0.0786,  0.0239, -0.1030,  0.0963,  0.0010, -0.0235, -0.0784, -0.0177,
          -0.0276,  0.1022, -0.0537, -0.1328,  0.1090, -0.0399,  0.0980,  0.0826],
         [ 0.0545, -0.0644,  0.0856, -0.1231,  0.0497,  0.0173,  0.0911,  0.0714,
           0.0283, -0.1396,  0.0759, -0.1161, -0.0204, -0.0179,  0.0262, -0.1240,
           0.1291, -0.1156, -0.0863, -0.0429, -0.0141,  0.0430,  0.0152, -0.0129,
          -0.1115,  0.0548,  0.0018,  0.0412,  0.0673,  0.1116,  0.1722,  0.0228,
        

In [18]:
print(enc_mod_features['ecg'].shape)
print(enc_mod_features['hr'].shape)

torch.Size([2, 64])
torch.Size([2, 64])
